In this notebook I am aiming to get all the data science related jobs available in Melbourne location from Seek.
All the matching skills will be displayed at the last.

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from xlsxwriter import workbook
import time
import re

After importing the libraries I have used for loop to iterate through first 20 pages of the job posting for Data analyst position and Melbourne and inner suburbs locations. This gives all the urls for the first 20 pages and requested the html_text to be read from requests.get(url) and read only text from it.

We are going to import details of each jobs with date posted, name of the company, position title, Link for the job posting and details of the jobs.

In [25]:
#This dataframe is created to store the jobs from first 20 pages of the Seek job board.
output_column_names = {'Company_name': str(), 'Job_title': str(), 'Job_Link': str(), 'Time_Posted': str(), 'Job Details': str()}
data1 = pd.DataFrame(output_column_names, index=[])

#Now comes the for loop to loop through first 20 pages of the webpage.
for i in range(1,20):
    url = 'https://www.seek.com.au/data-analyst-jobs/in-Melbourne-CBD-&-Inner-Suburbs-Melbourne-VIC?page={}'.format(i)
    html_text = requests.get(url).text
    #Now we will read all the html texts available in each page.
    soup = BeautifulSoup(html_text, 'html.parser')
    #Below is the class for each specific job postings and further reads will be done through that only.
    jobs = soup.find_all('div', class_ = 'yvsb870 v8nw070 v8nw072')
    data = []
    for job in jobs:
        #There are two featured jobs in each page with no date posting so we will remove those using the below lines.
        node = job.find(lambda tag: tag.name == 'span' and 
                                        tag.get('class') == ['yvsb870'])
        if node is not None:
            #Extraction of posting date is as below.
            date_posted = job.find(lambda tag: tag.name == 'span' and 
                                            tag.get('class') == ['yvsb870']).text
            #As this needs to be done for the new job postings we are now extracting data only which consists ago in their.
            #So, below filter will remove out job postings with dates such as 30th January which is like one month in the past and we do not want those postings.
            if re.search('ago', date_posted):
                #Now in the date_posted we will only extract text containing time of posting. Like 30m.
                date_posted = date_posted.split()
                date_posted = date_posted[0]
                #Extraction of company name is done through below.
                company_name = job.find('a', class_ = '_842p0a0').text
                #Now we will extract Job title using below code.
                Job_title = job.find('a', class_ = '_1tmgvw5 _1tmgvw7 _1tmgvwa _1tmgvwb _1tmgvwe yvsb870 yvsb87f _14uh994h').text
                #Below code is for Job Links extraction. So, one class does have lots of links available we only extract the job link for specific job.
                Job_link1 = []
                Job_link_Attr = job.find('h3', class_ = 'yvsb870 _14uh9944u _1cshjhy0 _1cshjhyl _1d0g9qk4 _1cshjhys _1cshjhy21')
                for link in Job_link_Attr.find_all('a'):
                    Job_link1.append(link.get('href'))
                Job_link = Job_link1[0]
                #Also, the job link is not in the required format. So, adding the headers at first.
                Job_link = 'https://www.seek.com.au' + Job_link
                #Details of the job is done through below code.
                Job_Skills = job.find_all('span', class_ = 'yvsb870 _14uh9944u _1cshjhy0 _1cshjhy1 _1cshjhy21 _1d0g9qk4 _1cshjhy7')
                Job_Skills1 = []
                for i in range(3,len(Job_Skills)):
                    Job_Skills1.append(Job_Skills[i].text)
                Job_Skills1 = '\n'.join(Job_Skills1)
                data.append([company_name, Job_title, Job_link, date_posted, Job_Skills1])
    data = pd.DataFrame(data, columns=output_column_names)
    #data1 = data1.append(data)
    data1 = pd.concat([data1, data])



Now that we have jobs for all the dates available we will now convert the job posting time to seconds and only save the jobs which are posted in recent 5 days and store the excel in the local folder.

In [27]:
#data2 = data1[data1["Time_Posted"].str.contains("h", "m", "s", "d")]

seconds_per_unit = {"s": 1, "m": 60, "h": 3600, "d": 86400, "w": 604800}

def convert_to_seconds(s):
    return int(s[:-1]) * seconds_per_unit[s[-1]]
#Second = []
#for i in data1:
#    try:
#        Second.append(data1["Time_Posted"][i].apply(convert_to_seconds))
data1["Seconds_Posted"] = data1["Time_Posted"].apply(convert_to_seconds)
data2 = data1.loc[data1['Seconds_Posted'] < 433000]
data2.to_excel("C:/Users/naiki/Downloads/Jobs1.xlsx", sheet_name="Results", engine='xlsxwriter', index = False)
